In [201]:
import pandas as pd

train = pd.read_csv("hr_train.csv")
test = pd.read_csv("hr_test.csv")

train.info()
# test.info()
# train.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  object 
 2   city_development_index  15326 non-null  float64
 3   gender                  11750 non-null  object 
 4   relevent_experience     15326 non-null  object 
 5   enrolled_university     15012 non-null  object 
 6   education_level         14961 non-null  object 
 7   major_discipline        13045 non-null  object 
 8   experience              15272 non-null  object 
 9   company_size            10539 non-null  object 
 10  company_type            10383 non-null  object 
 11  last_new_job            14984 non-null  object 
 12  training_hours          15326 non-null  int64  
 13  target                  15326 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

In [81]:
train.isnull().sum()

train.nunique()

train['target'].value_counts()

train.describe()
train.describe(include='object')

,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,15326,11750,15326,15012,14961,13045,15272,10539,10383,14984
unique,123,3,2,3,5,6,22,8,6,6
top,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1
freq,3510,10601,10942,11042,9242,11538,2673,2453,7838,6414


In [165]:
cols = train.select_dtypes(include='object').columns
# print(cols)
for col in cols:
    set_train = set(train[col])
    set_test = set(test[col])
    same = (set_train == set_test)
    if same:
        print(col, "\t카테고리 동일")
    else:
        print(col, "\t카테고리 동일하지 않음")

In [203]:
# 결측치 처리
train = train.fillna("X")
test = test.fillna("X")

# 데이터(변수) 선택
# target = train.pop('target')

X = train.drop(columns=['target'])
y = train['target']

# 레이블 인코딩
from sklearn.preprocessing import LabelEncoder
cols_train = train.select_dtypes(include='object').columns
print(cols_train)

cols_test = test.select_dtypes(include='object').columns
print(cols_test)

for col in cols_train:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])

for col in cols_test:
    le = LabelEncoder()
    test[col] = le.fit_transform(test[col])
train.info()
test.info()

Index(['city', 'gender', 'relevent_experience', 'enrolled_university',
       'education_level', 'major_discipline', 'experience', 'company_size',
       'company_type', 'last_new_job'],
      dtype='object')
Index(['city', 'gender', 'relevent_experience', 'enrolled_university',
       'education_level', 'major_discipline', 'experience', 'company_size',
       'company_type', 'last_new_job'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15326 entries, 0 to 15325
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             15326 non-null  int64  
 1   city                    15326 non-null  int32  
 2   city_development_index  15326 non-null  float64
 3   gender                  15326 non-null  int32  
 4   relevent_experience     15326 non-null  int32  
 5   enrolled_university     15326 non-null  int32  
 6   education_level         15326 non-null  int3

In [ ]:
# combined = pd.concat([train, test])
# cols = train.select_dtypes(include='object').columns
# for col in cols:
#     le = LabelEncoder()
#     combined[col] = le.fit_transform(combined[col])
# n_train = len(train)
# train = combined[:n_train]
# test = combined[n_train:]

In [209]:
# 스케일링
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
n_cols = train.select_dtypes(include='object').columns
train = scaler.fit_transform(train)
test = scaler.fit_transform(test)

In [211]:
# 훈련 데이터 분할(훈련용/검증용)
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=0)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((12260, 13), (3066, 13), (12260,), (3066,))

In [213]:
# 머신러닝 학습 및 평가
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# 모델 학습
rf = RandomForestClassifier(max_depth=7, n_estimators=200, random_state=0)
rf.fit(X_train, y_train)
pred = rf.predict_proba(X_test)
pred

# 성능 평가
roc_auc = roc_auc_score(y_test, pred)
print("roc_auc:", roc_auc)

ValueError: could not convert string to float: 'city_67'